# DEFRA dataset Find Missing Parts

- I will identify the missing values and data gaps in the DEFRA dataset and decide how to address them.
- I’ll start by importing the relevant modules and displaying the initial file paths.

In [ ]:
import pandas as pd
from pathlib import Path
import logging
from typing import Dict
import importlib.util

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# Use absolute path to avoid confusion
base_dir = Path("/Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels")
processed_dir = base_dir / "data" / "defra" / "processed"
metadata_path = base_dir / "data" / "defra" / "test" / "london_stations_clean.csv"

#new metadata path for parse and checking.
site_pollutant = base_dir / "data" / "defra" / "test" / "london_stations_clean.csv"
pollutant_mapping_path = base_dir / "src" / "data_prep" / "pollutant_mapps.py"

# Change output directory to data/defra/missing
missing_path = base_dir / "data" / "defra" / "missing"
missing_path.mkdir(parents=True, exist_ok=True)

## 1. Optimise for accurate file reading.
 - metadata_path shows london_stations_clean.csv
    - 2 row of the csv file: 
    - station_id,station_name,pollutant_available,pollutant_air,latitude,longitude,timeseries_id,pollutant
    - 785876,Borehamwood Meadow Park,Nitrogen dioxide,air,51.661229,-0.2705499999910774,4565.0,6875 - Borehamwood Meadow Park-Nitrogen dioxide (air)

- processed folder files: example: processed_dir path = /Borehamwood_Meadow_Park/NO__2023_01.csv
    2 row of the csv file:
    - timestamp,value,timeseries_id,station_name,pollutant_name,pollutant_std
    - 2023-02-01 00:00:00,0.125,4564,Borehamwood Meadow Park,Nitrogen monoxide,NO

metadata_paths and processed_dir file structure not matching. Solution need to add pollutant_std column to metadata_path file

- new structure of metadata_path:
    - station_id,station_name,pollutant_available,pollutant_std,pollutant_air,latitude,longitude,timeseries_id,pollutant
    - 785876,Borehamwood Meadow Park,Nitrogen dioxide,HERE STD VERSION OF POLLUTANT(NO2),air,51.661229,-0.2705499999910774,4565.0,6875 - Borehamwood Meadow Park-Nitrogen dioxide (air)

- standartise the pollutant names on pollutant_mapps.csv 
            -- DEFRA mappings common ones in both datasets first.
            'Nitrogen dioxide': 'NO2',
            'Nitrogen Dioxide': 'NO2',
            'Nitrogen_dioxide': 'NO2',
            'Nitric oxide': 'NO',
            'Nitrogen_monoxide': 'NO',
            'Nitrogen oxides': 'NOx',
            'Nitrogen_oxides': 'NOx',
            'PM2.5 Particulate': 'PM2.5',
            'Particulate_matter_less_than_2.5_micro_m': 'PM2.5',
            'PM10 Particulate': 'PM10',
            'Particulate_matter_less_than_10_micro_m': 'PM10',
            'Sulphur Dioxide': 'SO2',
            'Sulphur dioxide': 'SO2',
            'Sulphur_dioxide': 'SO2',
            'Ozone': 'O3',
            'Carbon Monoxide': 'CO',
            'Carbon monoxide': 'CO',
            'Carbon_monoxide': 'CO',
            
            -- VOCs used simplified standard codes instead of their chemical names.
            'Benzene': 'Benzene',
            'Toluene': 'Toluene',
            'Ethylbenzene': 'Ethylbenzene',
            'Ethyl_benzene': 'Ethylbenzene',
            'o-Xylene': 'o-Xylene',
            'm,p-Xylene': 'm,p-Xylene',
            
            -- Trimethylbenzenes.
            '1,2,3-Trimethylbenzene': '1,2,3-TMB',
            '1,2,4-Trimethylbenzene': '1,2,4-TMB',
            '1,3,5-Trimethylbenzene': '1,3,5-TMB',
            
            -- Alkanes.
            'Ethane': 'Ethane',
            'Propane': 'Propane',
            'n-Butane': 'n-Butane',
            'i-Butane': 'i-Butane',
            'n-Pentane': 'n-Pentane',
            'i-Pentane': 'i-Pentane',
            'n-Hexane': 'n-Hexane',
            'i-Hexane': 'i-Hexane',
            'n-Heptane': 'n-Heptane',
            'n-Octane': 'n-Octane',
            'i-Octane': 'i-Octane',
            
            - Alkenes
            'Ethene': 'Ethene',
            'Propene': 'Propene',
            '1-Butene': '1-Butene',
            'cis-2-Butene': 'cis-2-Butene',
            'trans-2-Butene': 'trans-2-Butene',
            '1-Pentene': '1-Pentene',
            'trans-2-Pentene': 'trans-2-Pentene',
            
            - Other VOCs
            '1,3-Butadiene': '1,3-Butadiene',
            '1.3_Butadiene': '1,3-Butadiene',
            'Isoprene': 'Isoprene',
            'Ethyne': 'Ethyne',

- New metadata file after add pollutant_std col std_london_sites_pollutant.csv and path below
    -  site_pollutant = base_dir / "data" / "defra" / "test" / "std_london_sites_pollutant.csv"

- Last std process for processed folder files:/defra/processed/2023measurements/Borehamwood_Meadow_Park/NO__2023_01.csv
    - add latitude,longitude coorditanetion columns.
    - timestamp,value,timeseries_id,station_name,pollutant_name,pollutant_std, latitude,longitude
    - latitude,longitude columns:
        - parse coordination columns from std_london_site_pollutant.csv
        - the way to parse it matching pollutant_std and station_name columns. 


 

### 1) Add column pollutant_std to std_london_sites_pollutant.csv file
- adding pollutant_std colmn to london_stations_clean.csv file (metadata_path).
    - new metadata_path saved as std_london_sites_pollutant.csv the path name changed as:
    - site_pollutant = base_dir / "data" / "defra" / "test" / "london_stations_clean.csv"

In [ ]:
# Add pollutant_std column to london_stations_clean.csv using DEFRA mappings
# and match it to pollutant_available column

defra_mappings = {
    'Nitrogen dioxide': 'NO2',
    'Nitrogen Dioxide': 'NO2',
    'Nitrogen_dioxide': 'NO2',
    'Nitric oxide': 'NO',
    'Nitrogen_monoxide': 'NO',
    'Nitrogen oxides': 'NOx',
    'Nitrogen_oxides': 'NOx',
    'PM2.5 Particulate': 'PM2.5',
    'Particulate_matter_less_than_2.5_micro_m': 'PM2.5',
    'PM10 Particulate': 'PM10',
    'Particulate_matter_less_than_10_micro_m': 'PM10',
    'Sulphur Dioxide': 'SO2',
    'Sulphur dioxide': 'SO2',
    'Sulphur_dioxide': 'SO2',
    'Ozone': 'O3',
    'Carbon Monoxide': 'CO',
    'Carbon monoxide': 'CO',
    'Carbon_monoxide': 'CO',
    'Benzene': 'Benzene',
    'Toluene': 'Toluene',
    'Ethylbenzene': 'Ethylbenzene',
    'Ethyl_benzene': 'Ethylbenzene',
    'o-Xylene': 'o-Xylene',
    'm,p-Xylene': 'm,p-Xylene',
    '1,2,3-Trimethylbenzene': '1,2,3-TMB',
    '1,2,4-Trimethylbenzene': '1,2,4-TMB',
    '1,3,5-Trimethylbenzene': '1,3,5-TMB',
    'Ethane': 'Ethane',
    'Propane': 'Propane',
    'n-Butane': 'n-Butane',
    'i-Butane': 'i-Butane',
    'n-Pentane': 'n-Pentane',
    'i-Pentane': 'i-Pentane',
    'n-Hexane': 'n-Hexane',
    'i-Hexane': 'i-Hexane',
    'n-Heptane': 'n-Heptane',
    'n-Octane': 'n-Octane',
    'i-Octane': 'i-Octane',
    'Ethene': 'Ethene',
    'Propene': 'Propene',
    '1-Butene': '1-Butene',
    'cis-2-Butene': 'cis-2-Butene',
    'trans-2-Butene': 'trans-2-Butene',
    '1-Pentene': '1-Pentene',
    'trans-2-Pentene': 'trans-2-Pentene',
    '1,3-Butadiene': '1,3-Butadiene',
    '1.3_Butadiene': '1,3-Butadiene',
    'Isoprene': 'Isoprene',
    'Ethyne': 'Ethyne',
}

# Load the stations metadata
stations_df = pd.read_csv(metadata_path)

# Add pollutant_std column by mapping pollutant_available
stations_df['pollutant_std'] = stations_df['pollutant_available'].map(defra_mappings)

# Save the updated DataFrame (commented out for now)
# stations_df.to_csv(metadata_path, index=False)

# Display first few rows to check
stations_df.head()

In [ ]:
def std_london_sites_pollutant(metadata_path, output_path, defra_mappings):
    """
    Adds a 'pollutant_std' column to the metadata and saves as std_london_sites_pollutant.csv.
    """
    df = pd.read_csv(metadata_path)
    df['pollutant_std'] = df['pollutant_available'].map(defra_mappings)
    df.to_csv(output_path, index=False)
    print(f"Saved standardised metadata to {output_path}")
    return df


## 2. Data Quality test Function:
The functions for discover and checks data quality metrics before cleaning, below.
- Counts total rows in dataset
- Identifies missing values per column (count + percentage)
- Counts duplicate rows based on timestamp
- Detects negative values in measurements
- Checks timestamp format issues


##### Defra dataset structure:
defra/processed/(year)_measurements/station_names(with empty spaces underscore)/(pollutant_std)_(year)_(month) 
 - first row of each csv file:
    - timestamp,value,timeseries_id,station_name,pollutant_name,pollutant_std

    



##### defra metadata: london_stations_clean.csv 
 - first row: station_id,station_name,pollutant_available,pollutant_air,latitude,longitude,timeseries_id,pollutant

In [ ]:
# Main data quality analysis function for DEFRA dataset
def defra_data_quality_analysis(processed_dir, pollutant_mapping_path, metadata_path, output_dir):
    """
    DEFRA Data Quality Analysis:
    - Checks empty files, missing columns, duplicates, missing values, types, format errors
    - Standardises pollutant names using mapping
    - Calculates issue rate for files with >20% missing 'value'
    """
    # Load pollutant mappings from python file as dictionary (assume dict named 'DEFRA_MAPPINGS' in file)
    

    # Load metadata for reference (not used in checks, but available)
    metadata = pd.read_csv(metadata_path)

    # Find all year directories (e.g. 2023_measurements)
    year_dirs = list(Path(processed_dir).glob('*_measurements'))
    missing_values_log = []
    all_issues = {
        'empty_files': [],
        'duplicate_timestamps': [],
        'high_missing_values': [],
        'column_errors': [],
        'format_errors': []
    }
    total_stats = {
        'total_files': 0,
        'files_processed': 0,
        'files_with_high_missing': 0,
        'total_rows': 0,
        'empty_files': 0
    }
    # Required columns for DEFRA files
    required_columns = ['timestamp', 'value', 'timeseries_id', 'station_name', 'pollutant_name', 'pollutant_std']

    # Traverse all year and station directories
    for year_dir in year_dirs:
        for station_dir in Path(year_dir).iterdir():
            if not station_dir.is_dir():
                continue
            for csv_file in station_dir.glob('*.csv'):
                total_stats['total_files'] += 1
                try:
                    df = pd.read_csv(csv_file)
                    # Check if file is empty
                    if df.empty:
                        all_issues['empty_files'].append(str(csv_file))
                        total_stats['empty_files'] += 1
                        continue
                    total_stats['files_processed'] += 1
                    total_stats['total_rows'] += len(df)

                    # Check for missing required columns
                    missing_cols = [col for col in required_columns if col not in df.columns]
                    if missing_cols:
                        all_issues['column_errors'].append({'file': str(csv_file), 'missing_columns': missing_cols})
                        continue

                    # Standardise pollutant names
                    df['pollutant_std'] = df['pollutant_name'].apply(lambda x: standardise_pollutant(x, mapping))

                    # Check for duplicate timestamps
                    dup_ts = df['timestamp'].duplicated().sum()
                    if dup_ts > 0:
                        all_issues['duplicate_timestamps'].append({'file': str(csv_file), 'duplicate_count': int(dup_ts)})

                    # Check missing values for each column (print summary)
                    for col in df.columns:
                        missing_count = df[col].isna().sum()
                        missing_pct = (missing_count / len(df) * 100) if len(df) > 0 else 0
                        logger.info(f"{csv_file.name}: Missing {col}: {missing_count} ({missing_pct:.2f}%)")

                    # Calculate missing value percentage for 'value' column
                    missing_values = df['value'].isna().sum()
                    empty_value_percentage = (100 * missing_values / len(df)) if len(df) > 0 else 0
                    logger.info(f"{csv_file.name}: Missing 'value': {missing_values}/{len(df)} ({empty_value_percentage:.2f}%)")
                    if empty_value_percentage > 20:
                        total_stats['files_with_high_missing'] += 1
                        missing_values_log.append({
                            'filename': csv_file.name,
                            'path': str(csv_file),
                            'station_name': df['station_name'].iloc[0] if 'station_name' in df.columns else '',
                            'pollutant_std': df['pollutant_std'].iloc[0] if 'pollutant_std' in df.columns else '',
                            'EmptyValuePercentage': round(empty_value_percentage, 2)
                        })
                except Exception as e:
                    all_issues['format_errors'].append({'file': str(csv_file), 'error': str(e)})

    # Calculate issue rate: percentage of processed files with >20% missing 'value'
    if total_stats['files_processed'] > 0:
        issue_rate = (total_stats['files_with_high_missing'] / total_stats['files_processed']) * 100
    else:
        issue_rate = 0.0
    print(f"\nIssue rate: {issue_rate:.2f}% of files have >20% missing 'value' column.")

    # Save log to CSV (commented out for now)
    # if missing_values_log:
    #     pd.DataFrame(missing_values_log).to_csv(Path(output_dir) / "logs_missin_value.csv", index=False)

    return all_issues, total_stats, issue_rate  # Return issue rate for inspection